In [6]:
import torch
from torch import nn
from d2l import torch as d2l
import my_utils

#VGG块
def vgg_block(num_convs,in_channels,out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

In [7]:
#VGG网络
conv_arch = ((1,64),(1,128),(2,256),(2,512),(2,512))

def vgg(conv_arch):
    conv_blks = []
    in_channels = 1
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(
            num_convs,in_channels,out_channels))
        in_channels = out_channels

    return nn.Sequential(
        *conv_blks,nn.Flatten(),
        nn.Linear(out_channels*7*7,4096),nn.ReLU(),
        nn.Dropout(0.5),nn.Linear(4096,4096),nn.ReLU(),
        nn.Dropout(0.5),nn.Linear(4096,10)
    )

net = vgg(conv_arch)

In [8]:
#观查每个层输出的形状

X = torch.randn(size=(1,1,224,224))
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__,'Output shape:',X.shape)

Sequential Output shape: torch.Size([1, 64, 112, 112])
Sequential Output shape: torch.Size([1, 128, 56, 56])
Sequential Output shape: torch.Size([1, 256, 28, 28])
Sequential Output shape: torch.Size([1, 512, 14, 14])
Sequential Output shape: torch.Size([1, 512, 7, 7])
Flatten Output shape: torch.Size([1, 25088])
Linear Output shape: torch.Size([1, 4096])
ReLU Output shape: torch.Size([1, 4096])
Dropout Output shape: torch.Size([1, 4096])
Linear Output shape: torch.Size([1, 4096])
ReLU Output shape: torch.Size([1, 4096])
Dropout Output shape: torch.Size([1, 4096])
Linear Output shape: torch.Size([1, 10])


In [ ]:
lr,num_epochs,batch_szie = 0.05,10,128
train_iter,test_iter = my_utils.load_data_fashion_mnist(batch_size=batch_szie,resize=224)
d2l.train_ch6(net,train_iter,test_iter,num_epochs,lr,d2l.try_gpu())

training on cuda:0
